In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy

import import_ipynb
import Pets_Loader
import Animals_Loader
import Marvel_Loader
import ResNetCaps_E

verbose = False
USE_CUDA = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CIFAR10_USE, MARVEL_USE, Animals_USE, PETS_USE = True, False, False, False

dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


batch_size = 32


if CIFAR10_USE: 
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if MARVEL_USE: 
    
    NUM_CLASSES = 26
    print("MARVEL")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"

    image_datasets = {'train': Marvel_Loader.MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': Marvel_Loader.MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")

if  PETS_USE:
    NUM_CLASSES = 37
    print("PETS")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Pets/Pet_Datasets"

    image_datasets = {'train': Pets_Loader.PETS_dataset(dat_file,train = True,transform=dataset_transform),'val': Pets_Loader.PETS_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    
    
if  Animals_USE:
    NUM_CLASSES = 50
    print("Animals")
    dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/Animals_with_Attributes2/JPEGImages"

    image_datasets = {'train': Animals_Loader.Animals_dataset(dat_file,train = True,transform=dataset_transform),'val': Animals_Loader.Animals_dataset(dat_file,train = False,transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")

    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")    


dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
model = ResNetCaps_E.ResNetCaps(NUM_CLASSES)
model = model.to(device)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 0.0001)

In [ ]:
loss_train = []        
accuracy_train = []

start = time.time()
n_epochs = 10

for epoch in range(n_epochs):
    print('epoch {}:{}'.format(epoch+1, n_epochs))     
    model.train() 
        
    train_loss = 0
    train_accuracy = 0

    for batch_id, (inputs, labels) in enumerate(dataloaders['train']):
        #inputs, labels = next(iter(dataloaders['train']))
        if MARVEL_USE: labels = labels-1
        labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
        inputs, labels = Variable(inputs), Variable(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = model.model_loss(outputs, labels)
        masked = model.decoder(outputs, inputs)

        loss.backward()
        optimizer.step()

        train_loss += loss.data
        train_accuracy += (sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))

        if batch_id % 100 == 0:
            print("train accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
            print("loss {}".format(loss.data))
            if verbose: print("masked {}".format(np.argmax(masked.data.cpu().numpy(), 1)))
            if verbose: print("labels {}".format(np.argmax(labels.data.cpu().numpy(), 1)))

    loss_train.append(train_loss/len(dataloaders['train']))
    accuracy_train.append(train_accuracy/len(dataloaders['train']))
    
end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy / len(dataloaders['train'])))
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_train, color='g')
plt.plot(epochs, accuracy_train, color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy - Loss')
plt.title('Training phase')
plt.show()

In [ ]:
model.eval()
test_loss = 0
test_accuracy = 0
start = time.time()
for batch_id, (inputs, labels) in enumerate(dataloaders['val']):
    if MARVEL_USE: labels = labels-1
    labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
    inputs, labels = Variable(inputs), Variable(labels)
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    masked = decoder(outputs, inputs)
    
    loss = model_loss(outputs, labels)
    test_loss += loss.data[0]
    test_accuracy += (sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))
    
    if batch_id % 100 == 0:
            print("test accuracy:", sum(np.argmax(masked.data.cpu().numpy(), 1) == 
                                   np.argmax(labels.data.cpu().numpy(), 1)) / float(batch_size))    
end = time.time()   
print("Validation time execution {}".format(end-start))
print("Loss value for test phase: {}".format(test_loss / len(dataloaders['val'])))
print("Accuracy value for test phase: {}".format(test_accuracy / len(dataloaders['val'])))


In [ ]:
torch.cuda.empty_cache()